# TEST DELL'ALGORITMO NON PARALLELIZZATO INTERAMENTE IN PYTORCH
# TUTTI I CASI SONO GESTITI CORRETTAMENTE (quindi è un ottimo punto di partenza!)

In [1]:
import torch
import numpy as np
import cvxpy as cp

def knapsack_specialized_pruning_complete(xi, v, w, C):

    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0
        if b != C - 1:
            b_list.append(int(b))
        if b + 1 > C - 1:
            break
            
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1

    if(w > v[-1]):
        x = torch.zeros(C)
        x[-1] = 1
    elif(w < v[0]):
        x = torch.zeros(C)
        x[0] = 1
    else:
        ratio = (xi/v)
        neg_indices = torch.where(ratio < 0)[0]
        neg_sorted = neg_indices[torch.argsort(ratio[neg_indices], descending=True)]
        pos_indices = torch.where(ratio >= 0)[0]
        pos_sorted = pos_indices[torch.argsort(ratio[pos_indices])]
        b_vector = np.concatenate([neg_sorted, pos_sorted])

        b = 0
        while(w/v[b_vector[b]] < 0 or x_plus[b_vector[b]] == 0):
            b += 1

        b_list_new = [b_vector[b]] 
        
        while(w/v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(w/v[b_vector[b]] < 0):
                continue
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
            if(len(b_list_new) > 2):
                b_list_new.pop(0)
                
        if(len(b_list_new) == 1 and w/v[b_list_new[0]] > 1):
            b_list_new[0] = 0
            
        x = torch.zeros(C)
        
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w/v[b_list_new[0]]), 5)
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 5)
            x[b_list_new[1]] = round(float(1 - theta), 5)
            
        objective_values = torch.matmul(x, xi)
    
    if(x is None): # Al caso di soluzione vuota faccio corrispondere un moltiplicatore vuoto 
        lambda_opt = None
    elif(torch.all(x == 0)): # Al caso di soluzione nulla faccio corrispondere un moltiplicatore nullo
        lambda_opt = 0
    elif(torch.count_nonzero(torch.abs(x) > 1e-6) == 2):
        i = torch.nonzero(torch.abs(x) > 1e-6, as_tuple=True) # i[0] è la lista [indice1, indice2]
        diff = i[0][1] - i[0][0]
        passo = v[1] - v[0] # Nel caso semplice sarebbe 1/C
        lambda_opt = round(float(-1 / (diff * passo) * (xi[i[0][1]] - xi[i[0][0]])), 5)
    elif(torch.count_nonzero(torch.abs(x) > 1e-6) == 1):
        i = torch.nonzero(torch.abs(x) > 1e-6, as_tuple=True)
        lambda_opt = round(-float(xi[i[0][0]] / v[i[0][0]]), 5) # i[0][0] è l'indice
    else:
        print("ERROR: SOLUTION NOT ACCEPTABLE!")

    objective_values = xi @ x
    return x, lambda_opt, objective_values, x_plus

# Dimension of vectors xi, v, and x
C = 5
M = 1

w0 = -0.01
r = 1.32
a = -2
b = 2.2
min_w, max_w = w0 - r, w0 + r

np.random.seed(41)  
torch.manual_seed(41)

conta_istanze_risolubili = 0
conta_soluzioni_sotto_soglia = 0
conta_moltiplicatori_errati = 0
conta_soluzioni_errate = 0

SUCCESS = 1

for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M) * (b - a) + a
    
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)

    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
    
    soglia = 0.0001
    if(x_pruning.value is not None):
        conta_istanze_risolubili += 1
        if(torch.max(x_opt) < soglia):
            conta_soluzioni_sotto_soglia += 1
    
    if(x_pruning.value is not None):
        if(not np.allclose(x_pruning.value, x_opt.numpy(), rtol=1e-5, atol=1e-2)):
            conta_soluzioni_errate += 1
            SUCCESS = 0
            print(f"{i} ERRORE NELLE SOLUZIONI:")
        if(np.abs(equality_constraint_pruning.dual_value[0] - lambda_opt) > 5e-2):
            conta_moltiplicatori_errati += 1
            SUCCESS = 0
            print(f"{i} ERRORE NEI MOLTIPLICATORI:")
            print(x_pruning.value)
    else: # da eliminare una volta verificato che i casi di soluzione non ammissibile sono gestiti correttamente
        print("x_opt:", x_opt)
        print("x_pruning.value:", x_pruning.value)
        print("lambda_opt:", lambda_opt)
        print("equality_constraint_pruning.dual_value:", equality_constraint_pruning.dual_value)
        print("-"*50)
            
if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")
    
print("\n\nconta_istanze_risolubili:", conta_istanze_risolubili)
print("conta_soluzioni_sotto_soglia:", conta_soluzioni_sotto_soglia)
print("conta_soluzioni_errate:", conta_soluzioni_errate)
print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)

/opt/anaconda3/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.0237
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.02873
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.06311
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.01949
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.20521
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.04973
equality_constraint_pruning.dual_value: None
---

x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.21006
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.12336
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.26775
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.02027
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.16132
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.0365
equality_constraint_pruning.dual_value: None
-----

x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.17589
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.00233
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.04257
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.2204
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.1734
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.29545
equality_constraint_pruning.dual_value: None
------

x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.15603
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.27756
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.0886
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.08045
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.22694
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.23374
equality_constraint_pruning.dual_value: None
-----

x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.01421
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.14586
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -0.72256
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -0.41871
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -0.58432
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.22706
equality_constraint_pruning.dual_value: None
---

x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.21014
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.22106
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.15039
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.2501
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -0.82209
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.20354
equality_constraint_pruning.dual_value: None
--

x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.13064
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.16899
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.11351
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.15689
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.19784
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.23051
equality_constraint_pruning.dual_value: None
---

x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.28321
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.13551
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.2449
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.26934
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.02165
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -0.93129
equality_constraint_pruning.dual_value: None
-----

x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.2501
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.14263
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.06907
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -0.68259
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([0., 0., 0., 0., 1.])
x_pruning.value: None
lambda_opt: -1.06792
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: tensor([1., 0., 0., 0., 0.])
x_pruning.value: None
lambda_opt: 0.06336
equality_constraint_pruning.dual_value: None
----

# CONFRONTO DEI TEMPI TRA L'ALGORITMO NON PARALLELIZZATO E CVXPY

In [2]:
import time

# Dimension of vectors xi, v, and x
C = 24
M = 1

w0 = -0.01
r = 1.32
a = -2
b = 2.2
min_w, max_w = w0 - r, w0 + r

np.random.seed(41)  
torch.manual_seed(41)

print("Start Specialized Algorithm")
start_time = time.time()
for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M) * (b - a) + a
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)
training_time = time.time() - start_time
print(f'Time taken by the specialized algorithm: {training_time:.2f} seconds')

print("Start cvxpy")
start_time = time.time()
for i in range(1000):
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
training_time2 = time.time() - start_time
print(f'Time taken by cvxpy: {training_time2:.2f} seconds')

print(f"\nThe Specialized Algorithm is {training_time2/training_time:.2f}X more perfomant than cvxpy")

Start Specialized Algorithm
Time taken by the specialized algorithm: 0.17 seconds
Start cvxpy
Time taken by cvxpy: 1.59 seconds

The Specialized Algorithm is 9.18X more perfomant than cvxpy


# TEST CON PARALLELIZZAZIONE

In [25]:
def knapsack_specialized_pruning_parallel(xi, v, w, C):
    M = w.shape[0]
    #print("xi:", xi)
    #print("v:", v)
    #print("w:", w, "\n")

    # Step 1: Calcolo x_plus (breakpoints)
    b_list = []
    b = 0
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0
        if b != C - 1:
            b_list.append(int(b))
        if b + 1 > C - 1:
            break
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    x_plus[torch.tensor(b_list)] = 1
    print("x_plus:", x_plus)

    # Preallocazioni
    x = torch.zeros(M, C)
    lambda_opt = torch.zeros(M)

    # Step 2: Classificazione dei problemi
    v0 = v[0]
    v_last = v[-1]
    mask_small = w < v0
    mask_large = w > v_last
    mask_mid = (~mask_small) & (~mask_large)
    #print("mask_small[71]:", mask_small[71])
    #print("mask_large[71]:", mask_large[71])
    #print("mask_mid[71]:", mask_mid[71])

    # CASO: w > v[-1]
    x[mask_large, -1] = 1

    # CASO: w < v[0]
    x[mask_small, 0] = 1
    #print("w[55]:", w[55])
    # CASO INTERMEDIO
    if mask_mid.any():
        M_mid = mask_mid.sum() #Numero di v[0]<w<v[-1]
        #print("M_mid:", M_mid)
        w_mid = w[mask_mid]
        #print("len(w_mid):", len(w_mid))
        #print("w_mid[55]:", w_mid[55])
        ratio = xi / v
        #print("ratio:", ratio)
        #print("ratio:", ratio)
        neg_indices = torch.where(ratio < 0)[0]
        neg_sorted = neg_indices[torch.argsort(ratio[neg_indices], descending=True)]
        pos_indices = torch.where(ratio >= 0)[0]
        pos_sorted = pos_indices[torch.argsort(ratio[pos_indices])]
        b_vector = torch.cat([neg_sorted, pos_sorted], dim=0) #questo è giusto
        print("b_vector:", b_vector)
        #print("w_mid[:, None][55]:", w_mid[:, None][55])
        ratio_b = w_mid[:, None] / v[b_vector]
        print("ratio_b[2]:", ratio_b[2])
        x_plus_b = x_plus[b_vector].bool()
        cond1 = (ratio_b >= 0) & x_plus_b
        #print("cond1[55]:", cond1[55])
        valid_i0 = cond1.float() * torch.arange(C)[None, :]
        valid_i0[~cond1] = float('inf')
        #print("valid_i0:", valid_i0)
        #print("valid_i0[55]:", valid_i0[55])
        i0_pos = valid_i0.argmin(dim=1)
        #print("i0_pos[55]:", i0_pos[55])
        
        i0 = b_vector[i0_pos]
        #print("i0[55]:", i0[55])

        v_i0 = v[i0]
        x_single = w_mid / v_i0
        #print("x_single:", x_single)
        invalid_i0 = x_plus[i0] == 0
        use_two = x_single > 1
        #print("use_two:", use_two)
        i1 = torch.full_like(i0, fill_value=-1)
        if use_two.any():
            b_vector_exp = b_vector.unsqueeze(0).expand(M_mid, -1)
            i0_exp = i0.unsqueeze(1).expand_as(b_vector_exp)
            x_plus_mask = x_plus[b_vector_exp] == 1
            greater_mask = b_vector_exp > i0_exp
            valid_mask = x_plus_mask & greater_mask
            masked_b_vector = torch.where(valid_mask, b_vector_exp, torch.full_like(b_vector_exp, C))
            i1_candidate, _ = masked_b_vector.min(dim=1)
            i1_candidate_use_two = i1_candidate[use_two]
            valid_i1_use_two = i1_candidate_use_two < C
            i0_use_two = i0[use_two]
            i1[use_two] = torch.where(valid_i1_use_two, i1_candidate_use_two, i0_use_two)
            #print("i1[55]:", i1[55])

        # Costruzione x_mid
        x_mid = torch.zeros(M_mid, C)

        # Caso: uso un solo indice
        mask_one = ~use_two
        rows_one = torch.where(mask_one)[0]
        cols_one = i0[mask_one]
        x_mid[rows_one, cols_one] = torch.clamp(w_mid[mask_one] / v[cols_one], 0.0, 1.0)

        # Caso: combinazione convessa
        mask_two = use_two & (i1 != i0)
        rows_two = torch.where(mask_two)[0]
        idx0 = i0[mask_two]
        idx1 = i1[mask_two]
        v0 = v[idx0]
        v1 = v[idx1]
        w_sel = w_mid[mask_two]
        theta = (w_sel - v1) / (v0 - v1)
        x_mid[rows_two, idx0] = torch.round(theta, decimals=5)
        x_mid[rows_two, idx1] = torch.round(1 - theta, decimals=5)

        x[mask_mid] = x_mid
        
    # === Calcolo vectorizzato dei moltiplicatori ===
    eps = 1e-6
    nz_mask = torch.abs(x) > eps
    nz_counts = nz_mask.sum(dim=1)
    #print("**** DENTRO LA FUNZIONE *****")
    lambda_opt = torch.zeros(x.shape[0])  # o torch.full(..., fill_value=torch.nan) se vuoi beccare errori

    # Caso 1 valore non nullo
    m1 = torch.where(nz_counts == 1)[0]
    #print("m1:", m1)
    if m1.numel() > 0:
        # Trova indici (m1_idx, col_idx) dove nz_mask è True, limitato a righe m1
        submask = nz_mask[m1]                     # shape: (len(m1), C)
        indices = submask.nonzero(as_tuple=False) # shape: (len(m1), 2)
        # estrai solo la colonna giusta
        i = indices[:, 1]
        lambda_opt[m1] = -xi[i] / v[i]
        lambda_opt[m1] = torch.round(lambda_opt[m1], decimals=5)


    # Caso 2 valori non nulli
    m2 = torch.where(nz_counts == 2)[0]
    #print("m2:", m2)
    if m2.numel() > 0:
        indices = nz_mask[m2].nonzero().reshape(-1, 2)
        grouped = indices.view(-1, 2, 2)
        i = grouped[:, 0, 1]
        j = grouped[:, 1, 1]
        delta_xi = xi[j] - xi[i]
        delta_idx = j - i
        passo = v[1] - v[0]
        #print("delta_xi:", delta_xi)
        #print("delta_idx:", delta_idx)
        #print("passo:", passo)
        lambda_opt[m2] = -delta_xi / (delta_idx * passo)
        lambda_opt[m2] = torch.round(lambda_opt[m2], decimals=5)
        #print("lambda_opt:", lambda_opt)

    #print("*"*40)
    objective_values = x @ xi
    return x, lambda_opt, objective_values, x_plus

# === MAIN SCRIPT ===
C = 10 #52
M = 10 #700
N = 10000

w0 = -0.11
r = 1.1
a = -1
b = 1
min_w, max_w = w0 - r, w0 + r

conta_istanze_risolubili = 0
conta_soluzioni_sotto_soglia = 0
conta_moltiplicatori_errati = 0
conta_soluzioni_errate = 0
SUCCESS = 1

for seed in [4402]:
    if(seed % 100 == 0):
        print("seed:", seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    xi_all = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w_all = torch.rand(M) * (b - a) + a

    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi_all, v, w_all, C)

    #if(list(b_vector) == list(range(C))):
    #    print("SEED:", seed)
        
    #print("\n\nx_opt:", x_opt, "\n")
    #print("w_all[55]:", w_all[55])
    for i in range(M):
        w = w_all[i].unsqueeze(0)
        x_i = cp.Variable(C)
        constraint = v @ x_i == w.item()
        constraints = [
            constraint,
            cp.sum(x_i) <= 1,
            x_i >= 0,
            x_i <= 1
        ]
        objective = cp.Minimize(xi_all.numpy() @ x_i)
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.ECOS)
        #print("x_i.value:", x_i.value)
        if x_i.value is not None:
            conta_istanze_risolubili += 1
            if torch.max(x_opt[i]) < 1e-4:
                conta_soluzioni_sotto_soglia += 1

            if not np.allclose(x_i.value, x_opt[i].numpy(), rtol=1e-5, atol=1e-2):
                print(f"{seed},{i} ERRORE NELLE SOLUZIONI:")
                print("xi_all:", xi_all)
                print("v:", v)
                print("w.item():", w.item())
                print("\nx_opt[i].numpy():", x_opt[i].numpy())
                print("x_i.value:", x_i.value)
                conta_soluzioni_errate += 1
                SUCCESS = 0

            dual_val = constraint.dual_value
            if dual_val is not None and abs(dual_val - lambda_opt[i].item()) > 5e-2:
                print(f"{seed},{i} ERRORE NEI MOLTIPLICATORI:\n\n\n")
                #print("x_plus:", x_plus)
                #print("b_vector:", b_vector)
                #print("SEED:", seed, "i:", i, "xi_all:", xi_all)
                #print("v:", v)
                #print("passo:", v[1]-v[0])
                #print("x_i.value:", x_i.value)
                #print("dual_val:", dual_val)
                #print("lambda_opt[i].item():", lambda_opt[i].item())
                #print()
                conta_moltiplicatori_errati += 1
                SUCCESS = 0

if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")
    
print("\nconta_istanze_risolubili:", conta_istanze_risolubili)
print("conta_soluzioni_sotto_soglia:", conta_soluzioni_sotto_soglia)
print("conta_soluzioni_errate:", conta_soluzioni_errate)
print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)

x_plus: tensor([1, 0, 1, 0, 0, 0, 1, 1, 1, 1], dtype=torch.int32)
b_vector: tensor([0, 1, 2, 3, 4, 5, 7, 8, 9, 6])
ratio_b[2]: tensor([-0.4711, -0.5757, -0.7402, -1.0363, -1.7272, -5.1817,  1.7272,  1.0363,
         0.7402,  5.1817])
4402,2 ERRORE NELLE SOLUZIONI:
xi_all: tensor([0.0214, 0.0562, 0.0803, 0.1482, 0.1789, 0.2241, 0.2304, 0.2788, 0.4711,
        0.8056])
v: tensor([-1.2100, -0.9900, -0.7700, -0.5500, -0.3300, -0.1100,  0.1100,  0.3300,
         0.5500,  0.7700])
w.item(): 0.569982647895813

x_opt[i].numpy(): [ 0.       0.       0.       0.       0.       0.       0.      -0.09083
  1.09083  0.     ]
x_i.value: [-2.74639501e-11 -2.76180986e-11 -2.30602550e-11 -1.89921500e-11
 -1.16783273e-11  3.12722034e-12  4.52254328e-11  2.81083646e-10
  9.09169577e-01  9.08304224e-02]
4402,2 ERRORE NEI MOLTIPLICATORI:




conta_istanze_risolubili: 8
conta_soluzioni_sotto_soglia: 0
conta_soluzioni_errate: 1
conta_moltiplicatori_errati: 1


# CONFRONTO DEI TEMPI TRA L'ALGORITMO PARALLELIZZATO E CVXPY

In [10]:
C = 32
M = 44000
N = 10

w0 = -0.01
r = 1.32
a = -2
b = 2.2
min_w, max_w = w0 - r, w0 + r

np.random.seed(41)  
torch.manual_seed(41)

print("Start Specialized Algorithm")
start_time = time.time()
for i in range(N):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M) * (b - a) + a
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi, v, w, C)
training_time = time.time() - start_time
print(f'Time taken by the specialized algorithm: {training_time:.2f} seconds')

print("Start cvxpy")
start_time = time.time()
for j in range(N):
    for i in range(M):
        # cvxpy solution with inequality constraint (pruning)
        x_pruning = cp.Variable(C)
        equality_constraint_pruning = v @ x_pruning == w[i]
        constraints_pruning = [
            equality_constraint_pruning,
            cp.sum(x_pruning) <= 1,
            x_pruning >= 0,
            x_pruning <= 1
        ]
        objective_pruning = cp.Minimize(xi @ x_pruning)
        prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
        prob_pruning.solve(solver=cp.SCS)
training_time2 = time.time() - start_time
print(f'Time taken by cvxpy: {training_time2:.2f} seconds')

print(f"\nThe Specialized Algorithm is {training_time2/training_time:.2f}X more perfomant than cvxpy")

Start Specialized Algorithm
Time taken by the specialized algorithm: 0.10 seconds
Start cvxpy
Time taken by cvxpy: 839.42 seconds

The Specialized Algorithm is 8236.91X more perfomant than cvxpy
